# Mean-Reversion

## Data Cleaning

Prior to diving into the strategy itself, we will gather the required data. I am using the Coingecko API as my Velodata access is limited and I am not able to get the required 365 days historic data.

In [19]:
import polars as pl
import pycoingecko

cg = pycoingecko.CoinGeckoAPI()

coin_list = ['aptos', 'arbitrum', 'avalanche-2', 'binancecoin', 'bitcoin-cash', 'bitcoin', 'bonk', 'cardano', 'celestia', 'chainlink', 'cosmos', 'dogecoin', 'dogwifcoin', 'ethereum', 'floki', 'injective-protocol', 'litecoin', 'matic-network', 'near', 'notcoin', 'optimism', 'pepe', 'polkadot', 'ripple', 'shiba-inu', 'solana', 'sui', 'the-open-network', 'thorchain', 'tron']
load = 0

if load:
    for coin in coin_list:
        data = cg.get_coin_market_chart_by_id(coin, 'usd', 365)
        df_coin = (pl.DataFrame(data['prices'], strict=False)
                .transpose()
                .with_columns(pl.lit(coin).alias('ticker'), 
                                pl.all().cast(pl.String))
                )
        df_coin.write_csv(f"./data/{coin}.csv")

Next, we combine and clean up the dataframe.

In [20]:
try:
    df = (
        pl.scan_csv(f"./data/*.csv", infer_schema_length=0).with_columns(
            pl.from_epoch(pl.col('column_0').cast(pl.Float32), time_unit='ms').alias('timestamp'),
            pl.col('column_1').cast(pl.Float32).alias('price')
        ).drop(
            ['column_0', 'column_1']
        )
    )
except pl.exceptions.ComputeError as e:
    raise(e)

df = df.collect()

# round the timestamp
df = df.with_columns(
    pl.col('timestamp').dt.date()
).sort(['timestamp'])

## Exploratory Data Analysis

### Data Gaps

Next, we will take a look at tokens where a significant amount of data is missing. Unsurprisingly, Notcoin, DogWifHat and Celestia do not have a full year of data.

In [21]:
df.group_by('ticker').agg(pl.count()).sort('count').head(10)

C:\Users\vosko\AppData\Local\Temp\ipykernel_17292\138382080.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  df.group_by('ticker').agg(pl.count()).sort('count').head(10)


ticker,count
str,u32
"""notcoin""",87
"""dogwifcoin""",243
"""celestia""",286
"""litecoin""",365
"""dogecoin""",365
"""polkadot""",365
"""solana""",365
"""bitcoin""",365
"""injective-protocol""",365


### Strategy

The strategy that we will be looking at is pair-trading which will make use of longing and shorting an asset at the same time. Essentially, we are trying to long/short the spread between the two assets. As a start, we will look at correlations amongst the prices.

In [23]:
# pivot table
df.pivot(index='timestamp', values='price', on='ticker', aggregate_function='first')

ComputeError: found multiple elements in the same group, please specify an aggregation function